<center>
    <img src="https://www.uni-magdeburg.de/pub/theme/ovgu_min_theme/skin/img/logo_std.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


#### Importing Needed packages


In [ ]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup as bsoup
import time
import csv
import re

##### Months conversion: German to English

In [ ]:
def germanMonthsToEnglish(gMonth):
    if gMonth.lower() == 'januar':
        return 'January'
    if gMonth.lower() == 'februar':
        return 'February'
    if gMonth.lower() == 'märz':
        return 'March'
    if gMonth.lower() == 'april':
        return 'April'
    if gMonth.lower() == 'mai':
        return 'May'
    if gMonth.lower() == 'juni':
        return 'June'
    if gMonth.lower() == 'juli':
        return 'July'
    if gMonth.lower() == 'august':
        return 'August'
    if gMonth.lower() == 'september':
        return 'September'
    if gMonth.lower() == 'oktober':
        return 'October'
    if gMonth.lower() == 'november':
        return 'November'
    if gMonth.lower() == 'dezember':
        return 'December'

#### Step 1: To get the source code of the review page

- #####  get type of industry from csv file
- #####  get url and company name from csv file

In [ ]:
rev_pages=1000
count_final=0

# open and fetch csv
companyListFile = 'dataOCM//200429_Companies_List.csv'
companyName = ''
companyUrl = ''
companyType = ''
rev_org_dict = {}
rev_org_type_dict = {}
with open(companyListFile, encoding='utf-8') as companyDataFile:
    data = csv.reader(companyDataFile, delimiter=',')
    for row in data:
        if row[6]=='Yes':
            companyName = row[1]
            companyUrl = row[3]
            companyType = row[4]
            rev_org_dict.update({companyName: companyUrl})
            rev_org_type_dict.update({companyName: companyType})

In [ ]:
#initializing variables

rev_org_domain=''
OrgSales= ''
OrgNoOfEmployees= ''
OrgKununuScore= ''
OrgTotalKununuReviews= ''
OrgRecomPercent= ''
OrgProfileViews= ''

##### Framework of output files

In [ ]:
masterDataFileName = 'Master_Data_Milestone1_test.csv'
masterDataFile = open(masterDataFileName, "w", newline='',encoding='utf-8')
csv_out = csv.writer(masterDataFile, delimiter = '|')
csv_out.writerows([("Org","OrgSector","RverMonthYear","RverReviewer","RverPosition","RverLoc","RverRecom","RvReviewAbout","RvScore","RvComment")])

orgDataFileName = 'Orgs_Data_Milestone1.csv'
orgDatafile = open(orgDataFileName, "w", newline='',encoding='utf-8')
csv_out = csv.writer(orgDatafile, delimiter = '|')
csv_out.writerows([("Org","OrgSector","OrgSales","OrgNoOfEmployees","OrgKununuScore","OrgTotalKununuReviews","OrgRecomPercent","OrgProfileViews","OrgBenefits")])



##### Data extraction from html tags

:warning:

### We extracted the data from kununu.com on 03/06/2020. The below will be outdated if kununu updates its layout

####  Step 2: To get the Review part from kununu page

In [ ]:
base_url='https://www.kununu.com/de/'
komment_url='/kommentare/'
#base_url='https://www.kununu.com/de/deutschebahn/kommentare/'
#base_url='https://www.kununu.com/de/infosyslimited/kommentare/'

for Org, org_url_alias in rev_org_dict.items():
    time.sleep(0.5)
    passFlag = False  
    print('Fetching Reviews of', Org, ',Please wait...')
    OrgSector = rev_org_type_dict[Org]
    url_org_home=base_url+org_url_alias
    try:
        page_org_home=urlopen(url_org_home)
    except:
        continue
    html_org_home=page_org_home.read()
    page_org_home.close()
    parsed_html_org=bsoup(html_org_home,"html.parser")
#    rev_org_domain=parsed_html_org.find("div",{"class":"company-profile-sub-title"}).a.text # Domain
    key_fig_div=parsed_html_org.find_all("div",{"class":"col-xs-7 col-sm-12 col-md-12 col-lg-12 company-profile-number-data"})

    try: # OrgSales
        OrgSalesTemp = key_fig_div[0].text.strip().split(' ')
        num = ''
        unit = ''
        currency = ''
        for t in OrgSalesTemp:
            if t[0].isnumeric():
                num = t
            if "mio" in t.lower():
                unit = 'million'
            if "mrd" in t.lower():
                unit = 'billion'
            if "usd" in t.lower():
                currency = 'usd'
            if "eur" in t.lower():
                currency = 'eur'
        newnum = ''
        for t in range(len(num)):
            if num[t] == '.':
                newnum = newnum + ''
            elif num[t] == ',':
                newnum = newnum + '.'
            else:
                newnum = newnum + num[t]
        finalVal = 0
        if unit == 'million':
            finalVal = float(newnum) / 1000
            if currency == 'usd':
                finalVal = finalVal * 0.9
        if unit == 'billion':
            finalVal = float(newnum)
            if currency == 'usd':
                finalVal = finalVal * 0.9
        OrgSales=str(str(finalVal))
    except:
        OrgSales= ''

    try: # OrgNoOfEmployees
        OrgNoOfEmployeesTemp=key_fig_div[1].text.strip().split(' ')
        num = []
        for t in OrgNoOfEmployeesTemp:
            if t[0].isnumeric():
                num.append(t)
        newnum = []
        for a in num:
            newnumTemp = ''
            for t in range(len(a)):
                if a[t] == '.':
                    newnumTemp = newnumTemp + ''
                elif a[t] == ',':
                    newnumTemp = newnumTemp + '.'
                else:
                    newnumTemp = newnumTemp + a[t]
            newnum.append(newnumTemp)
        finalVal = 0
        for a in newnum:
            finalVal = finalVal + int(a)

        OrgNoOfEmployees=str(finalVal)
    except:
        OrgNoOfEmployees= ''
    kununu_dtls=parsed_html_org.find("div",{"class":"col-sm-5 col-md-7 overview-main"}).div
    try:
        OrgKununuScore=kununu_dtls.div.span.text.strip().replace('.', '').replace(',', '.')
    except:
        OrgKununuScore= ''
    try:
        OrgRecomPercent=kununu_dtls.find("div", {"class": "col-xs-6 col-sm-6 col-md-3 col-lg-3 relative"}).a.span.text.strip().replace('%','').replace('.', '').replace(',', '.')
    except:
        OrgRecomPercent= ''
    try:
        OrgProfileViews=kununu_dtls.find("div", {"class": "col-md-2 col-lg-2 hidden-sm hidden-xs relative"}).a.span.text.strip().replace('.', '').replace(',', '.')
    except:
        OrgProfileViews= ''
    try:
        OrgTotalKununuReviews=parsed_html_org.find_all("div", {"class": "base-comparison"})[0].text.strip().split(' ')[0].replace('.', '').replace(',', '.') # Total reviews in kununu
    except:
        OrgTotalKununuReviews= ''
    try:
        OrgBenefits = ''
        regex = re.compile('.*cp-tile company-profile-benefits.*')
        size = len(parsed_html_org.find_all("div", {"class": regex})[0].div.find_all("benefit"))
        for p in range(size):
            OrgBenefits = OrgBenefits + " (" + str(p+1) + ") " + re.sub(' +', ' ',parsed_html_org.find_all("div", {"class": regex})[0].div.find_all("benefit")[p].text.strip())
    except:
        OrgBenefits= ''

    f_csv_list_org = []
    f_csv_list_org.append((Org, OrgSector, OrgSales, OrgNoOfEmployees, OrgKununuScore, OrgTotalKununuReviews,
                       OrgRecomPercent, OrgProfileViews, OrgBenefits))
    csv_out = csv.writer(orgDatafile, delimiter='|')
    csv_out.writerows(f_csv_list_org)

    for page in range(1, rev_pages):
        time.sleep(0.5)
        target_url = base_url +org_url_alias+komment_url+str(page)
        try:
            page_obj = urlopen(target_url)  # Object of HTTPResponse from the url
        except HTTPError:
            print("404 Page not found") 
            passFlag = True  # Check it later for all 
            break
        html_page = page_obj.read()  # Open the HTTPResponse object to get the html source code.
        page_obj.close()  # Close the HTTPResponse object

        # Step 2: To get the Review part from kununu page
        parsed_page = bsoup(html_page, "html.parser")  # html_page is parsed in bs4 readable format
        all_divs = parsed_page.find_all("div", {"class": "index__reviewBlock__27gnB"})  # Got all review frames of different reviewers
        count_div = 0
        userCounter = 1
        for indiv_divs in all_divs:
            count_div += 1
            try:
                engMonth = germanMonthsToEnglish(indiv_divs.div.span.time.text.split(' ')[0])
                RverMonthYear = engMonth + " " + indiv_divs.div.span.time.text.split(' ')[1]  # extracted datetime from the time tag
            except:
                RverMonthYear= ''
            try:
                RvComment1 = indiv_divs.div.h3.get_text(separator=" ").replace('\n', ' ').replace('\r', ' ')
            except:
                RvComment1= ''
            try:
                RvScore1 = indiv_divs.find("div", {"class": "index__block__36tsj index__scoreBlock__138n3"}).span.text.replace('.', '').replace(',', '.')
            except:
                RvScore1= ''
            # Employee details
            try:
                RverPosition = indiv_divs.find("span", {"class": "index__position__mCyeO"}).text.replace('\n', ' ').replace('\r', ' ') 
            except:
                RverPosition= ''
            try:
                RverLoc = indiv_divs.find("span", {"class": "index__sentence__3PKUg index__middot__3vlu3"}).text  # Emp department with location
            except:
                RverLoc= ''
            try:
                RverRecom = indiv_divs.find("span", {"class": "index__recommendation__jftd3"}).text.replace('\n', ' ').replace('\r', ' ') 
            except:
                RverRecom= ''


            # defining all variables
            # Ratings
            RvReviewAbout2 = ''
            RvScore2 = ''
            RvComment2 = ''
            RvReviewAbout3 = ''
            RvScore3 = ''
            RvComment3 = ''
            RvReviewAbout4 = ''
            RvScore4 = ''
            RvComment4 = ''
            RvReviewAbout5 = ''
            RvScore5 = ''
            RvComment5 = ''
            RvReviewAbout6 = ''
            RvScore6 = ''
            RvComment6 = ''
            RvReviewAbout10 = ''
            RvScore10 = ''
            RvComment10 = ''

            RvReviewAbout7 = 'Corona1' # Corona 1 # Wofür möchtest du deinen Arbeitgeber im Umgang mit der Corona-Situation loben?
            RvComment7 = ''
            RvReviewAbout8 = 'Corona2' # Corona 2 # Wo siehst du Chancen für deinen Arbeitgeber mit der Corona-Situation besser umzugehen?
            RvComment8 = ''
            RvReviewAbout9 = 'Corona3' # Corona 3 # Wie kann dich dein Arbeitgeber im Umgang mit der Corona-Situation noch besser unterstützen?
            RvComment9 = ''

            #    list_diversity=['Arbeitsatmosphäre','Kollegenzusammenhalt','Gleichberechtigung','Umgang mit älteren Kollegen','Umwelt-/Sozialbewusstsein']
            rev_internalAttributes = indiv_divs.find_all("div", {"class": "index__factor__3Z15R"})
            for rev_class in rev_internalAttributes:
                if ('Wofür möchtest du deinen Arbeitgeber' in rev_class.h4.text):
                    try:
                        RvComment7 = rev_class.p.get_text(separator=" ").replace('\n', ' ').replace('\r', ' ')
                    except:
                        RvComment7 = ''
                if ('Wo siehst du Chancen' in rev_class.h4.text):
                    try:
                        RvComment8 = rev_class.p.get_text(separator=" ").replace('\n', ' ').replace('\r', ' ') 
                    except:
                        RvComment8 = ''
                if ('Wie kann dich dein Arbeitgeber' in rev_class.h4.text):
                    try:
                        RvComment9 = rev_class.p.get_text(separator=" ").replace('\n', ' ').replace('\r', ' ') 
                    except:
                        RvComment9 = ''
                if (rev_class.h4.text == 'Arbeitsatmosphäre'):
                    try:
                        RvReviewAbout2 = rev_class.h4.text
                    except:
                        RvReviewAbout2= ''
                    try:
                        RvScore2 = rev_class.span["data-score"].replace('.', '').replace(',', '.')
                    except:
                        RvScore2= ''
                    try:
                        RvComment2 = rev_class.p.get_text(separator=" ").replace('\n', ' ').replace('\r', ' ') 
                    except:
                        RvComment2 = ''
                if (rev_class.h4.text == 'Kollegenzusammenhalt'):
                    try:
                        RvReviewAbout3 = rev_class.h4.text
                    except:
                        RvReviewAbout3= ''
                    try:
                        RvScore3 = rev_class.span["data-score"].replace('.', '').replace(',', '.')
                    except:
                        RvScore3= ''
                    try:
                        RvComment3 = rev_class.p.get_text(separator=" ").replace('\n', ' ').replace('\r', ' ') 
                    except:
                        RvComment3 = ''
                if (rev_class.h4.text == 'Gleichberechtigung'):
                    try:
                        RvReviewAbout4 = rev_class.h4.text
                    except:
                        RvReviewAbout4= ''
                    try:
                        RvScore4 = rev_class.span["data-score"].replace('.', '').replace(',', '.')
                    except:
                        RvScore4= ''
                    try:
                        RvComment4 = rev_class.p.get_text(separator=" ").replace('\n', ' ').replace('\r', ' ') 
                    except:
                        RvComment4 = ''
                if (rev_class.h4.text == 'Umgang mit älteren Kollegen'):
                    try:
                        RvReviewAbout5 = rev_class.h4.text
                    except:
                        RvReviewAbout5= ''
                    try:
                        RvScore5 = rev_class.span["data-score"].replace('.', '').replace(',', '.')
                    except:
                        RvScore5= ''
                    try:
                        RvComment5 = rev_class.p.get_text(separator=" ").replace('\n', ' ').replace('\r', ' ') 
                    except:
                        RvComment5 = ''
                if (rev_class.h4.text == 'Umwelt-/Sozialbewusstsein'):
                    try:
                        RvReviewAbout6 = rev_class.h4.text
                    except:
                        RvReviewAbout6= ''
                    try:
                        RvScore6 = rev_class.span["data-score"].replace('.', '').replace(',', '.')
                    except:
                        RvScore6= ''
                    try:
                        RvComment6 = rev_class.p.get_text(separator=" ").replace('\n', ' ').replace('\r', ' ') 
                    except:
                        RvComment6 = ''
                if (rev_class.h4.text == 'Work-Life-Balance'):
                    try:
                        RvReviewAbout10 = rev_class.h4.text
                    except:
                        RvReviewAbout10= ''
                    try:
                        RvScore10 = rev_class.span["data-score"].replace('.', '').replace(',', '.')
                    except:
                        RvScore10= ''
                    try:
                        # RvComment10= rev_class.p.text.replace('\n', ' ').replace('\r', ' ') 
                        RvComment10 = rev_class.p.get_text(separator=" ").replace('\n', ' ').replace('\r', ' ')
                    except:
                        RvComment10 = ''

            f_csv_list = []
            f_csv_list.append((Org, OrgSector, RverMonthYear, str(userCounter),
                                    RverPosition, RverLoc, RverRecom, "Overall", RvScore1, RvComment1))
            f_csv_list.append((Org, OrgSector, RverMonthYear, str(userCounter),
                                    RverPosition, RverLoc, RverRecom, RvReviewAbout2, RvScore2, RvComment2))
            f_csv_list.append((Org, OrgSector, RverMonthYear, str(userCounter),
                                    RverPosition, RverLoc, RverRecom, RvReviewAbout3, RvScore3, RvComment3))
            f_csv_list.append((Org, OrgSector, RverMonthYear, str(userCounter),
                                    RverPosition, RverLoc, RverRecom, RvReviewAbout4, RvScore4, RvComment4))
            f_csv_list.append((Org, OrgSector, RverMonthYear, str(userCounter),
                                    RverPosition, RverLoc, RverRecom, RvReviewAbout5, RvScore5, RvComment5))
            f_csv_list.append((Org, OrgSector, RverMonthYear, str(userCounter),
                                    RverPosition, RverLoc, RverRecom, RvReviewAbout6, RvScore6, RvComment6))
            f_csv_list.append((Org, OrgSector, RverMonthYear, str(userCounter),
                               RverPosition, RverLoc, RverRecom, RvReviewAbout7, "NA", RvComment7))
            f_csv_list.append((Org, OrgSector, RverMonthYear, str(userCounter),
                               RverPosition, RverLoc, RverRecom, RvReviewAbout8, "NA", RvComment8))
            f_csv_list.append((Org, OrgSector, RverMonthYear, str(userCounter),
                               RverPosition, RverLoc, RverRecom, RvReviewAbout9, "NA", RvComment9))
            f_csv_list.append((Org, OrgSector, RverMonthYear, str(userCounter),
                               RverPosition, RverLoc, RverRecom, RvReviewAbout10, RvScore10, RvComment10))
            userCounter = userCounter + 1
            csv_out = csv.writer(masterDataFile, delimiter='|')
            csv_out.writerows(f_csv_list)

        count_final = count_div + count_final
    if passFlag == True: 
        passFlag = False 
        pass 
    print('Total Reviews Fetched Until Now :: ', count_final,'\n')
